In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('takehome_users.csv',encoding = "latin")
df2 = pd.read_csv('takehome_user_engagement.csv')

In [3]:
df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [5]:
df['creation_time'] = pd.to_datetime(df['creation_time'])

In [6]:
df2.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [8]:
df2['user_id'].nunique()

8823

In [9]:
df['object_id'].nunique()

12000

In [10]:
df2['time_stamp'] = pd.to_datetime(df2['time_stamp'])
df2.index = df2['time_stamp']
df2 = df2.drop('time_stamp',axis=1)

As a reminder, 'Defining an "adopted user" as a user who has logged into the product on three separate
days in at least one sevenday period, identify which factors predict future user
adoption.'

In [11]:
#Group by user_id, resample to 1 week period, and sum visits over period
df2_agg = df2.groupby([pd.Grouper(freq='W'),'user_id']).sum()

In [12]:
df2_agg[df2_agg.visited>=3].head()

visited
time_stamp user_id         
2012-06-10 1693           3
2012-06-17 728            4
           11764          3
2012-06-24 1693           3
           5297           3

In [13]:
#Create a dataframe of all users with 3 or more visits in a week period
df_adopt = df2_agg[df2_agg.visited>=3].unstack(level=1).melt()
adopted_users = pd.DataFrame(df_adopt.user_id.unique(),columns=['user_id'])

In [14]:
adopted_users

,user_id
0,1693
1,728
2,11764
3,5297
4,6171
...,...
1440,7868
1441,7927
1442,9870
1443,10746


In [15]:
df_merge = df.merge(adopted_users,how='inner',left_on='object_id',right_on='user_id')
df_merge.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id
0,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2
1,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0,10
2,20,2014-03-06 11:46:38,Helms Mikayla,lqyvjilf@uhzdq.com,SIGNUP,1.401364e+09,0,0,58,NaN,20
3,33,2014-03-11 06:29:09,Araujo José,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,1.401518e+09,0,0,401,79.0,33
4,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1.401045e+09,1,0,235,NaN,42


In [16]:
df_merge[df_merge['user_id']==0]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id


In [17]:
#we can drop user personal info
df_merge = df_merge.drop(['object_id','creation_time','name','email','user_id'],axis=1)

In [18]:
df_merge.isna().sum()

creation_source                 0
last_session_creation_time      0
opted_in_to_mailing_list        0
enabled_for_marketing_drip      0
org_id                          0
invited_by_user_id            628
dtype: int64

In [19]:
#we do not need to know 'invited_by_user_id' for ever user, and user_id = 0 does not exist, so we can have a placeholder
df_merge['invited_by_user_id']=df_merge['invited_by_user_id'].fillna(value=0)

In [20]:
df_merge

,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,ORG_INVITE,1.396238e+09,0,0,1,316.0
1,ORG_INVITE,1.401833e+09,1,1,318,4143.0
2,SIGNUP,1.401364e+09,0,0,58,0.0
3,GUEST_INVITE,1.401518e+09,0,0,401,79.0
4,SIGNUP,1.401045e+09,1,0,235,0.0
...,...,...,...,...,...,...
1440,GUEST_INVITE,1.400657e+09,0,0,65,11251.0
1441,GUEST_INVITE,1.401524e+09,0,0,15,5688.0
1442,GUEST_INVITE,1.401411e+09,1,1,52,6647.0
1443,GUEST_INVITE,1.400757e+09,1,0,31,6410.0


In [21]:
df_merge['creation_source'].nunique()

5

In [22]:
#one hot encode creation_source feature
df_merge=pd.get_dummies(df_merge,['creation_source'])

In [23]:
#convert columns to float64
for col in df_merge.columns:
    df_merge[col] = df_merge[col].astype('float64')
df_merge.head()

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
0,1.396238e+09,0.0,0.0,1.0,316.0,0.0,1.0,0.0,0.0,0.0
1,1.401833e+09,1.0,1.0,318.0,4143.0,0.0,1.0,0.0,0.0,0.0
2,1.401364e+09,0.0,0.0,58.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.401518e+09,0.0,0.0,401.0,79.0,1.0,0.0,0.0,0.0,0.0
4,1.401045e+09,1.0,0.0,235.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#scale data
scaler = StandardScaler()
features = scaler.fit_transform(df_merge)

#fit PCA
pca = PCA()
components = pca.fit_transform(features)

In [35]:
df_comp = pd.DataFrame(pca.components_,columns=df_merge.columns,index=['PC-1','PC-2','PC-3','PC-4','PC-5','PC-6','PC-7','PC-8','PC-9','PC-10'])
#absolute values of correlation with principal components
best_features = np.absolute(df_comp[np.absolute(df_comp) > 0.1])
best_features.head()

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
PC-1,NaN,NaN,NaN,NaN,0.637653,0.232628,0.486575,0.233976,0.390324,0.296733
PC-2,NaN,0.695712,0.69891,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PC-3,0.116892,NaN,NaN,NaN,NaN,0.784060,0.590037,NaN,NaN,NaN
PC-4,NaN,NaN,NaN,0.261476,NaN,NaN,NaN,0.108246,0.648711,0.697782
PC-5,0.230407,NaN,NaN,0.219055,NaN,NaN,NaN,0.825688,0.296054,0.350811


In [36]:
df_comp.sum(axis=0).sort_values(ascending=False)

enabled_for_marketing_drip            1.484239
creation_source_GUEST_INVITE          0.566233
opted_in_to_mailing_list              0.059848
invited_by_user_id                   -0.175063
creation_source_ORG_INVITE           -0.188481
creation_source_PERSONAL_PROJECTS    -0.204505
org_id                               -0.365326
creation_source_SIGNUP_GOOGLE_AUTH   -0.670668
last_session_creation_time           -1.831408
creation_source_SIGNUP               -1.851357
dtype: float64

In [37]:
#sum correlations to get a relative estimate of the feature importance
best_features.sum(axis=0).sort_values(ascending=False)

creation_source_SIGNUP_GOOGLE_AUTH    2.375500
creation_source_SIGNUP                2.270659
creation_source_PERSONAL_PROJECTS     2.019568
creation_source_ORG_INVITE            1.959578
org_id                                1.792770
creation_source_GUEST_INVITE          1.782990
last_session_creation_time            1.700941
enabled_for_marketing_drip            1.405251
opted_in_to_mailing_list              1.401398
invited_by_user_id                    1.400076
dtype: float64

From PCA, we can see that the first 7 principal components account for most of the variation in adopted users. The top 3 features pertain to creation sources and are most predictive for adoptive users.